# 主題:用RNN做情意分析
我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」

## 1. 初始準備
基本上和之前是一樣的, 我們就不再說明。

In [15]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 2. 讀入 IMDB 電影數據庫
今天我們要評入 IMDB 電影數據庫影評的部份。

In [17]:
from keras.datasets import imdb

In [18]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)   # num_words=10000 代表只要最常出現的一萬個字

In [19]:
len(x_train)

25000

In [20]:
len(x_test)

25000

## 2.1 輸入資料部份
我們來看一下輸入部份長什麼樣子?

In [25]:
x_train[99]   # 頻率最常出現的字為1

[1,
 1230,
 3765,
 566,
 97,
 189,
 102,
 86,
 7,
 32,
 4,
 973,
 16,
 55,
 355,
 18,
 14,
 20,
 4,
 64,
 542,
 173,
 16,
 4,
 893,
 2115,
 5376,
 250,
 39,
 8013,
 4,
 1362,
 2,
 14,
 102,
 47,
 57,
 599,
 633,
 6,
 1317,
 2,
 8,
 6,
 189,
 20,
 57,
 206,
 57,
 116,
 5,
 57,
 836,
 82,
 6,
 1317,
 2,
 3728,
 2,
 9,
 6,
 52,
 284,
 21,
 29,
 9,
 38,
 2245,
 5,
 1044,
 11,
 14,
 15,
 45,
 619,
 50,
 71,
 6,
 171,
 531,
 15,
 71,
 424,
 8,
 30,
 163,
 6211,
 4,
 1629,
 189,
 212,
 102,
 5,
 57,
 31,
 1498,
 11,
 4,
 311,
 13,
 197,
 15,
 14,
 20,
 16,
 1150,
 1479,
 5,
 13,
 161,
 990,
 692,
 5,
 1706,
 12,
 69,
 77,
 1194,
 8,
 3245,
 2001,
 553,
 67,
 14,
 20,
 48,
 25,
 423,
 13,
 131,
 124,
 51,
 25,
 122,
 236,
 1506,
 198,
 4,
 64,
 552,
 7,
 415,
 37,
 62,
 169,
 14,
 20,
 60,
 2602,
 629,
 5,
 615,
 14,
 9,
 8,
 25,
 1230,
 3765,
 570,
 231,
 189,
 102,
 14,
 20,
 166,
 2039,
 168,
 40,
 2450,
 5486,
 3298]

In [27]:
for i in range(10):
    print(len(x_train[i]), end=', ') # 顯示前10筆影評資料的長度

218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 

## 2.2 輸出資料部份
輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [28]:
y_train[15]

0

In [29]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

## 2.3 送入神經網路的輸入處理
雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [31]:
from keras.preprocessing import sequence

In [32]:
# 整理sequence的長度 = sequence.pad_sequences(你的sequence,maxlen=sequence的最長長度)

x_train = sequence.pad_sequences(x_train, maxlen=100)  # 看到第100個字便要決定是正評或是負評，若不足100個字keras會補 0上去
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [33]:
x_train.shape

(25000, 100)

## 3. 打造你的 RNN
這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

## 3.1 決定神經網路架構
* 先將 10000 維的文字壓到 128 維
* 然後用 150 個 LSTM
* 最後一個 output, 直接用 sigmoid 送出

## 3.2 建構我們的神經網路
文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少(10000), 還有我們打算壓到幾維 (128)。

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Embedding   # Embedding 將 10000維的文字壓到 128維
from keras.layers import LSTM

In [35]:
model = Sequential()

In [36]:
model.add(Embedding(10000, 128))   # Embedding層可以把離散或整數的資料變得連續

In [37]:
model.add(LSTM(150))

In [38]:
model.add(Dense(1, activation='sigmoid'))

## 3.3 組裝
這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [39]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               167400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 3*(128+150+1)*150=125550  1為biases ; 3為控制LSTM的3個閘門
# (128+150+1)*150=41850
# 125550 + 41850=167400

In [40]:
model.compile(loss='binary_crossentropy',   # binary_crossentropy做二元分類常用
            optimizer='adam',
            metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## 4. 訓練
我們用的Embedding中,會被batch_size影響輸出。

輸入的shape會是(batch_size,每筆上限),

也就是(32,100)輸出是(32,100,128),其中128是我們決定是壓成幾維的向量

In [41]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=5)




Epoch 1/5





25000/25000 [==============================] - 274s 11ms/step - loss: 0.4274 - acc: 0.7998
Epoch 2/5
25000/25000 [==============================] - 338s 14ms/step - loss: 0.2702 - acc: 0.8914
Epoch 3/5
25000/25000 [==============================] - 323s 13ms/step - loss: 0.1982 - acc: 0.9244
Epoch 4/5
25000/25000 [==============================] - 918s 37ms/step - loss: 0.1419 - acc: 0.9481
Epoch 5/5
25000/25000 [==============================] - 680s 27ms/step - loss: 0.1092 - acc: 0.9595


## 5. 檢視結果
## 5.1 分數
我們照例來看看測試資料的分數。

In [44]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 61s 2ms/step


In [45]:
print('測試資料的loss', score[0])
print('測試資料的正確率', score[1])

測試資料的loss 0.46386356464385986
測試資料的正確率 0.82068


## 5.2 儲存結果
這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。

In [46]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)
model.save_weights('imdb_model_weights.h5')

另一種存的方式

In [47]:
model.save('myrnn.h5')